In [1]:
import pandas as pd
import requests
import re
import pickle

In [10]:
def get_number_of_GZ():
  r =requests.get("https://www.cuitonline.com/sitemap.xml")
  xml_file=str(r.text)
  return xml_file.count('sitemap-detalle-')

In [2]:
#### VA DESDE 1 A 680 A LA FECHA DE 19 OCT-2021
direccion_cuit1="https://www.cuitonline.com/sitemaps/sitemap-detalle-"
direccion_cuit2=".xml.gz"
MAX_GZ = get_number_of_GZ()

In [3]:
i=1 ### 4 minutos 11seg en descargar todo
while i <= MAX_GZ:
  url= direccion_cuit1 + str(i) + direccion_cuit2
  file_name_1="sitemap-detalle-"
  file_name_2=".txt.gz"
  f=file_name_1 + str(i) +file_name_2
  r = requests.get(url, allow_redirects=True)
  open(f, 'wb').write(r.content)
  i=i+1

KeyboardInterrupt: ignored

In [ ]:
!mkdir unzips

In [ ]:
!find . -name "*txt.gz" -exec mv "{}" ./unzips \;

In [ ]:
!find . -name "*txt.gz" | xargs gunzip 

In [ ]:
%cd unzips

In [ ]:
!ls

In [ ]:
#cat *.txt > datos_cuit.txt

In [ ]:
n=1
f1="sitemap-detalle-"
f2=".txt"
while n<= MAX_GZ:
  f=f1+str(n)+f2
  file = open(f,encoding = 'unicode_escape')
  s = file.read()
  file.close()
  result = re.findall('https://www.cuitonline.com/detalle/(.*).html', s)
  arch_limpio="CUITS___"+str(n)+f2
  with open(arch_limpio, "w") as outfile:
    outfile.write("\n".join(result))
  n=n+1

In [ ]:
len(result)

In [ ]:
print(result)

In [ ]:
n=1
f1="CUITS___"
f2=".txt"
df_final = pd.DataFrame(
    {
        "CUIT": ["0"],

        "NOMBRE": ["XYZ"],
    },
    index=[0],
)
while n<= MAX_GZ:
  f=f1+str(n)+f2
  data = pd.read_csv(f, sep="/", header=None)
  data.columns = ["CUIT", "NOMBRE"]
  df_final = pd.concat([df_final,data])
  n+=1

In [ ]:
df_final

In [ ]:
df_final.CUIT=df_final.CUIT.astype({'CUIT': 'int64'})

In [ ]:
df_final.dtypes

In [ ]:
df_final.to_csv('DB_CUIT_22OCT21.csv',sep='/')

In [ ]:
df_final

In [ ]:
!rm *.txt

In [ ]:
df_final

In [ ]:
df_empresas=df_final[(df_final.CUIT)>=30000000000]

In [ ]:
df_empresas

In [ ]:
df_empresas.to_csv('DB_CUIT_EMPRESAS_22OCT21.csv',sep='/')

In [ ]:
df_personas=df_final[(df_final.CUIT)<=30000000000]

In [ ]:
df_personas

In [ ]:
df_personas.to_csv('DB_CUIT_PERSONAS_22OCT21.csv',sep='/')

In [ ]:
df_personas_h=df_personas[df_personas.CUIT<=21000000000]

In [ ]:
df_personas_h ### PERSONAS MASCULINAS

In [ ]:
df_personas_h.to_csv('DB_CUIT_PERSONAS_HOMBRE_22OCT21.csv',sep='/')

In [ ]:
df_personas_m=df_personas[(df_personas.CUIT<=28000000000) & (df_personas.CUIT>=27000000000)]

In [ ]:
df_personas_m

In [ ]:
df_personas_m.to_csv('DB_CUIT_PERSONAS_MUJER_22OCT21.csv',sep='/')

In [ ]:
df_personas_REP=df_personas[(df_personas.CUIT<=24000000000) & (df_personas.CUIT>=23000000000)]

In [ ]:
df_personas_REP

In [ ]:
df_personas_REP.to_csv('DB_CUIT_PERSONAS_REPETIDOS_22OCT21.csv',sep='/')

In [ ]:
df_personas_h[(df_personas_h.DNI>=36807330)]

In [ ]:
def get_DNI(CUIT):### AB12345678X
  STR_CUIT=str(CUIT)
  STR_DNI= STR_CUIT[2] + STR_CUIT[3] +STR_CUIT[4] +STR_CUIT[5] +STR_CUIT[6] +STR_CUIT[7] +STR_CUIT[8] +STR_CUIT[9]
  return int(STR_DNI)

In [ ]:
df_personas=df_personas.drop(0)

In [ ]:
df_personas.head(5)

In [ ]:
df_personas["DNI"]=df_personas["CUIT"].apply(get_DNI)

In [ ]:
df_personas